In [1]:
%pip install sktime==0.11.3
#%pip install numpy=1.20

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump

from sktime.classification.distance_based import ElasticEnsemble
from sktime._contrib.vector_classifiers._rotation_forest import RotationForest
from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []
def gpu(X, y, kf):
  with tf.device('/device:GPU:0'):
    i= 0
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)    
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train.head())
        #print(type(X_train))
        print("Treining EE...")
        EE =     ElasticEnsemble(
        proportion_of_param_options=0.1,
        proportion_train_for_test=0.1,
    )
        print(f"Interaction {i}")
        print("In trainning...")
        EE.fit(X_train, y_train)
        print("In prediction...")
        i=i+1
        
        # Predict and print accuracy
        predictions = EE.predict(X_test)
        precision.append(precision_score(y_test, predictions, average='macro',zero_division=1))
        accuracy.append(accuracy_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions, average='macro',zero_division=1))
        f1.append(f1_score(y_test, predictions, average='macro'))
        #print("In prediction...")
        # Predict and print accuracy
       # predictions = HC2.predict(X_test)
    print(type(X_train))
    print(np.shape(X_train))
    print(f"Precision: {np.mean(precision)}, Accuracy:{np.mean(accuracy)}, Recall: {np.mean(recall)}, F1: {np.mean(f1)}")
    return np.mean(precision), np.mean(accuracy), np.mean(recall), np.mean(f1), np.std(accuracy), EE

(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("Using MirroredStrategy")
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  print("Using Default Strategy")
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))
  precision, accuracy, recall, f1, std, EE = gpu(X,y,kf)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Treining EE...
Interaction 0
In trainning...


D:\Pessoais\AnacondaFiles\envs\teste_with_gpu\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Pessoais\AnacondaFiles\envs\teste_with_gpu\lib\site-packages\numba\core\caching.py", line 482, in save
    data_name = overloads[key]
KeyError: ((array(float64, 2d, C), array(float64, 2d, C)), ('x86_64-pc-windows-msvc', 'skylake', '+64bit,+adx,+aes,-amx-bf16,-amx-int8,-amx-tile,+avx,+avx2,-avx512bf16,-avx512bitalg,-avx512bw,-avx512cd,-avx512dq,-avx512er,-avx512f,-avx512ifma,-avx512pf,-avx512vbmi,-avx512vbmi2,-avx512vl,-avx512vnni,-avx512vp2intersect,-avx512vpopcntdq,+bmi,+bmi2,-cldemote,+clflushopt,-clwb,-clzero,+cmov,+cx16,+cx8,-enqcmd,+f16c,+fma,-fma4,+fsgsbase,+fxsr,-gfni,+invpcid,-lwp,+lzcnt,+mmx,+movbe,-movdir64b,-movdiri,-mwaitx,+pclmul,-pconfig,-pku,+popcnt,-prefetchwt1,+prfchw,-ptwrite,-rdpid,+rdrnd,+rdseed,-r

In [ ]:
df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','std'])
df_metrics['precision'] = precision
df_metrics['accuracy'] = accuracy
df_metrics['recall'] = recall
df_metrics['f1_score'] = f1
df_metrics['std'] = std
df_metrics

In [ ]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv("results/" + 'df_metrics_EE.csv', index=False)
#save the model
print("Saving model...")
dump(EE, open('results/'+'EE_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

In [ ]:
#if __name__ == "__main__":
    # Load data
def gpu():
  with tf.device('/device:GPU:0'):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score'])
    X_train, y_train = load_arff("AllCandidasCollectReduced.arff")
    #X_test, y_test = load_arff("AllCandidas_TEST_2.arff")
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    precision = make_scorer(precision_score, average='weighted',zero_division=1)
    recall = make_scorer(recall_score, average='weighted',zero_division=1)
    print(type(X_train))
    print(type(y_train))
    print(np.shape(X_train))
    print(np.shape(y_train))
    # Fit HC2
    print("Treining EE...")
    EE =     ElasticEnsemble(
    proportion_of_param_options=0.1,
    proportion_train_for_test=0.1,
)
    print("In prediction...")
    res['precision'] = cross_val_score(EE, X_train, y_train, cv=cv,scoring=precision).mean()
    print(f"Precision result: {res['precision'][0]}")
    res['accuracy'] = cross_val_score(EE, X_train, y_train, cv=cv, scoring='accuracy').mean()
    print(f"Accuracy result: {res['accuracy'][0]}")
    res['recall'] = cross_val_score(EE, X_train, y_train, cv=cv,scoring=recall).mean()
    print(f"Recall result: {res['recall'][0]}")
    res['f1_score'] = cross_val_score(EE, X_train, y_train, cv=cv,scoring='f1_weighted').mean()
    print(f"F1_score result: {res['f1_score'][0]}")
    return res , EE
df_metrics, EE_model = gpu()
df_metrics

C:\Users\mlb\AppData\Local\Temp/ipykernel_30112/1082374303.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(19999, 1)
(19999,)
Treining EE...
In prediction...


In [ ]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv('results/'+ 'df_metrics_EE.csv', index=False)
#save the model
print("Saving model...")
dump(EE_model, open('results/'+'EE_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

In [ ]:
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))